In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model

img_a_inp = Input((28, 28), name='img_a_inp')
img_b_inp = Input((28, 28), name='img_b_inp')


def get_cnn_block(depth):
    return Sequential([Conv2D(depth, 3, 1),
                       BatchNormalization(),
                       ReLU()])


DEPTH = 64
cnn = Sequential([Reshape((28, 28, 1)),
                  get_cnn_block(DEPTH),
                  get_cnn_block(DEPTH * 2),
                  get_cnn_block(DEPTH * 4),
                  get_cnn_block(DEPTH * 8),
                  GlobalAveragePooling2D(),
                  Dense(64, activation='relu')])

feature_vector_A = cnn(img_a_inp)
feature_vector_B = cnn(img_b_inp)

concat = Concatenate()([feature_vector_A, feature_vector_B])

dense = Dense(64, activation='relu')(concat)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[img_a_inp, img_b_inp], outputs=output)

model.summary()

In [ ]:
import numpy as np

random_indices = np.random.choice(X_train.shape[0], 30, replace=False)

X_train_sample, y_train_sample = X_train[random_indices], y_train[random_indices]

X_train_sample.shape, y_train_sample.shape

In [ ]:
len(X_train_sample) ** 2

In [ ]:
import itertools


def make_paired_dataset(X, y):
    X_pairs, y_pairs = [], []

    tuples = [(x1, y1) for x1, y1 in zip(X, y)]

    for t in itertools.product(tuples, tuples):
        pair_A, pair_B = t
        img_A, label_A = t[0]
        img_B, label_B = t[1]

        new_label = int(label_A == label_B)

        X_pairs.append([img_A, img_B])
        y_pairs.append(new_label)

    X_pairs = np.array(X_pairs)
    y_pairs = np.array(y_pairs)

    return X_pairs, y_pairs

In [ ]:
make_paired_dataset(X_train_sample, y_train_sample)

In [ ]:
X_train_pairs, y_train_pairs = make_paired_dataset(X_train_sample, y_train_sample)

X_train_pairs.shape, y_train_pairs.shape

In [ ]:
random_indices = np.random.choice(X_test.shape[0], 15, replace=False)

X_test_sample, y_test_sample = X_test[random_indices], y_test[random_indices]

X_test_sample.shape, y_test_sample.shape

In [ ]:
X_test_pairs, y_test_pairs = make_paired_dataset(X_test_sample, y_test_sample)

X_test_pairs.shape, y_test_pairs.shape

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3)

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard

call = [TensorBoard(log_dir='logs', histogram_freq=1, write_images=True)]
# tensorboard --logdir=logs

In [ ]:
model.fit(x=[X_train_pairs[:, 0, :, :], X_train_pairs[:, 1, :, :]],
          y=y_train_pairs,
          validation_data=([X_test_pairs[:, 0, :, :],
                            X_test_pairs[:, 1, :, :]],
                           y_test_pairs),
          epochs=1000,
          batch_size=64,
          callbacks=call)

In [ ]:
model.save('model')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model')

In [ ]:
img_A, img_B = X_test[0], X_test[8]
label_A, label_B = y_test[0], y_test[8]

label_A, label_B

In [ ]:
import matplotlib.pyplot as plt

plt.figure(dpi=28)
plt.imshow(img_A)

In [ ]:
plt.figure(dpi=28)
plt.imshow(img_B)

In [ ]:
model.predict([img_A.reshape((1, 28, 28)),
               img_B.reshape((1, 28, 28))]).flatten()[0] > 0.5